In [ ]:
# ! ~/looktranslators-master/lv2look p5483WGFracNSPPosc

In [ ]:
# lv2look version: 28.10.2016


# Reading file p5483WGFracNSPPosc: 24-bit File has 115998608 bytes of data, 2230736 recs, 13 chans
# First recs are:
# 	time, ch2, ch3, ch4, ch5, ch6, ch7, ch8, ch9, ch10, ch11, ch12, ch13
# 	10000, 2808365, 8638786, 381164, 845528, 740133, -144442, -109736, 3898134, -467673, 4242348, 3937852, 2763713  

# Experiment p5483WGFracNSPPosc  began at 4:22 PM  on 1/31/21 
# n_chans = 13, nrecs = 2230736
# Channel 2 (Vert DCDT   ) was recorded at +/- 10.0 volts
# Channel 3 (Vert LOAD   ) was recorded at +/- 10.0 volts
# Channel 4 (Hor DCDT    ) was recorded at +/- 10.0 volts
# Channel 5 (Hor LOAD    ) was recorded at +/- 10.0 volts
# Channel 6 (Pc Disp     ) was recorded at +/- 10.0 volts
# Channel 7 (Pc LOAD     ) was recorded at +/- 10.0 volts
# Channel 8 (Ppa Disp    ) was recorded at +/- 10.0 volts
# Channel 9 (Ppa LOAD    ) was recorded at +/- 10.0 volts
# Channel 10 (Ppb Disp    ) was recorded at +/- 10.0 volts
# Channel 11 (Ppb LOAD    ) was recorded at +/- 10.0 volts
# Channel 12 (Int Disp    ) was recorded at +/- 10.0 volts
# Channel 13 (Sync        ) was recorded at +/- 10.0 volts

In [ ]:
from pathlib import Path
from pylook.units import units
import pylook.calc as lc
from pylook.io import read_binary
import numpy as np
import pandas as pd

In [ ]:
# data_path1 = 'p5483WGFracNSPPoscl'
# data_path2 = 'p5483WGFracNSPPosc_part2l'

# data1, _ = read_binary(data_path1)
# data2, _ = read_binary(data_path2)

# data1 = pd.DataFrame.from_dict(data1)
# data2 = pd.DataFrame.from_dict(data2)

# data = data1.append(data2)
# np.savez('p5384_mechdat', data)
# data.to_csv('p5384_mechdat.csv')

# data = np.load('p5384_mechdat.npz')['arr_0']
data = pd.read_csv('p5384_mechdat.csv') 

In [ ]:
data['Time'] = np.cumsum(data['Time']/10000)

data['norm disp'] = data.pop('Hor DCDT')
data['norm stress'] = data.pop('Hor LOAD')
data['pc disp'] = data.pop('Pc Disp')
data['pc press'] = data.pop('Pc LOAD')
data['ppa disp'] = data.pop('Ppa LOAD')
data['ppa press'] = data.pop('Ppa Disp')
data['ppb disp'] = data.pop('Ppb LOAD')
data['ppb press'] = data.pop('Ppb Disp')

In [ ]:
calibration_file_path = './calibrations.csv'
calibrations = pd.read_csv(calibration_file_path)

def get_gain(load, hilo):
    gain = calibrations[calibrations['Type'].str.contains(load, case=False) & 
       calibrations['Gain'].str.contains(hilo, case=False)].values[0,4]
    
    return gain


def get_dcdt_gain(rodLen, hilo, HV='horizontal dcdt'):
    gain = calibrations[calibrations['Type'].str.contains(HV, case=False) & 
              calibrations['Rod Length'].str.contains(rodLen, case=False) & 
              calibrations['Gain'].str.contains(hilo, case=False)].values[0,4]
    
    return gain

In [ ]:
# CALIBRATIONS FOR DISPLACEMENTS AND STRESSES
#---------------------------------------------------------------------------------------

# HORIZONTAL DISPLACEMENT
# high gain, short rod: 0.6438 mm/V
h_disp_calib = (20/2**24)  * get_dcdt_gain('short', 'high') * 1000 #um/mm 
data['norm disp'] = data['norm disp'] * h_disp_calib

# HORIZONTAL LOAD
# low gain: 123.9 mV/kN
# 1 surface
# area: 0.002233036 m^2
h_load_calib = 1 * 0.0022292545 * get_gain('44mm Solid Horiz', 'high') * (2**24/20)
data['norm stress'] = data['norm stress'] * 1/h_load_calib

#---------------------------------------------------------------------------------------

# Pc INTENSIFIER DISPLACEMNT
# high gain: 29.499 mm/V 
# ((20/2**24) * units('V / bit')) * (29.499 * units('mm / V') * 1000 * units('micron / mm'))
pc_disp_calib = (20/2**24) * get_gain('pc_lvdt', 'none') * 1000 #um/mm
data['pc disp'] = data['pc disp'] * pc_disp_calib

# Pc PRESSURE
# high gain: 0.14709 V/MPa
pc_pres_calib = (20/2**24)  * 1/(get_gain('pc_trdx', 'none'))
data['pc press'] = data['pc press'] * pc_pres_calib

#---------------------------------------------------------------------------------------
                                                     
# PpA INTENSIFIER DISPLACEMNT
# high gain: 26.882 mm/V 
ppa_disp_calib = (20/2**24) * get_gain('ppa lvdt', 'none') * 1000 #um/mm
data['ppa disp'] = data['ppa disp'] * ppa_disp_calib

# PpA PRESSURE
# high gain: 1.5118 V/MPa
ppa_pres_calib = (20/2**24) * 1/(get_gain('ppa trdx', 'high'))
data['ppa press'] = data['ppa press'] * ppa_pres_calib
                               
#---------------------------------------------------------------------------------------
                                                      
# PpB INTENSIFIER DISPLACEMNT
# high gain: 26.882 mm/V 
ppb_disp_calib = (20/2**24) * get_gain('ppb lvdt', 'none') * 1000 #um/mm
data['ppb disp'] = data['ppb disp'] * ppb_disp_calib

# PpB PRESSURE
# high gain: 1.48 V/MPa
ppb_pres_calib = (20/2**24) * 1/(get_gain('ppb trdx', 'high'))
data['ppb press'] = data['ppb press'] * ppb_pres_calib
                                                      
#---------------------------------------------------------------------------------------                                                      

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot, row, column

output_notebook()

def make_runplot(data, x_var='Time', y_vars=None, tools='pan,box_zoom, xzoom_in, yzoom_in, reset,save,box_select,hover'):
    plots = []
    for col_name in list(data):
        if col_name == x_var:
            continue
        if y_vars and (col_name not in y_vars):
            continue
        
        # First plot is simple, the rest we share the x range with the first
        if plots == []:
            p = figure(title=col_name, tools=tools)
        else:
            p = figure(title=col_name, tools=tools, x_range=plots[0].x_range)
        
        # Plot the data and set the labels
#         p.xaxis.axis_label = str(data[x_var].units)
#         p.yaxis.axis_label = str(data[col_name].units)
        p.line(data[x_var], data[col_name])
        
        plots.append(p)
    show(gridplot(plots, ncols=1, plot_width=1200, plot_height=400))



In [ ]:
data = data.drop(data.index[1337300:1397100])

In [ ]:
# make_runplot(data, y_vars=['ppa press', 'ppb press'])

In [ ]:
# OFFSETS AND ZERO POINTS

# HOR DISP
data['norm disp'] = lc.zero(data['norm disp'], 341, mode='before')

# HOR LOAD
data['norm stress'] = lc.zero(data['norm stress'], 359, mode='before')

# PC DISP
data['pc disp'] = lc.zero(data['pc disp'], 6539, mode='before')

# PC LOAD
data['pc press'] = lc.zero(data['pc press'], 6541, mode='before')

# Ppa LOAD
data['ppa press'] = lc.zero(data['ppa press'], 15110, mode='before')

# Ppb LOAD
data['ppb press'] = lc.zero(data['ppb press'], 20094, mode='before')


In [ ]:
# make_runplot(data, y_vars=['Hor DCDT', 'Hor LOAD'])

In [ ]:
##################################################################################
# cjm & cew; updated for True Triax config. 20181031                             #
#                                                                                #
#                                                                                #
#   In the vessel for SDS with notched L-shaped samples:                         #
#               Shear Stress is given by: tau = (FVp - Pc*Ap)/A                  #
#               Normal stress is given by: sigma_n = (FHp + Pc(A-Ap))/A          #
#                    or, sigma_n = FHp/A + Pc(A-Ap)/A                            #
#  where:                                                                        #
#  FVp is vertical force applied by the piston                                   #
#  FHp is horizontal force applied by the piston                                 #
#  A is the smaller area of the L-shaped samples                                 #
#   note that the frictional contact area will be given by the block thickness.  #
#  The eventual fracture plane will be given by the distance between the notches #
#  Ap is the area of the piston                                                  #
#  Pc is the confining pressure                                                  #
#                                                                                #
##################################################################################


# Account for Pc Force pushing on pistons
# Horizontal Area = 0.0022292545 m^2
A = 0.0022292545
#adjust normal stress for Pc.
#Area of piston, Ap = 44mm (dia) =  0.00152053084 m^2
Ap = 0.044 * np.pi
Pc_area = 100 - (Ap-A)/A
# ~ 32% of Pc is added to the applied horizontal stress to get the effective stress

# Calculate Effective Stresses
data['effNS'] = data['norm stress'] + (data['pc press'] * Pc_area/100) - (data['ppa press'] + data['ppb press'])/2

In [ ]:
# make_runplot(data, y_vars=['Hor DCDT', 'effNS'])

In [ ]:
from perm_funcs import movingslope, movingmean

###############################################################################
# VOLUMETRIC FLOW                                                             #
# first, calculate displacement rate of the pistons with running avg. slope   #
# then, multiply by inner diameter of intensifiers for volumetric rate        #
###############################################################################

# sampfreq = 100;

# idxfs100only = np.where(np.logical_and(sampfreq > 90, sampfreq < 110))

# data['PpDiff'][idxfs100only];

###############################################################################

# calculate Pressure differential & convert from [MPa] --> [Pa]
data['PpDiff'] = (data['ppa press'] - data['ppb press']) * 1e6 #Pa

data['mmPpDiff'] = movingmean(data['PpDiff'],101);

# INTENSIFIER DISPLACEMENT RATES
data['AvPparate'] = movingslope(data['ppa disp'], 101)
data['AvPpbrate'] = movingslope(data['ppb disp'], 101)

# INTENSIFIER VOLUME
piston_dia = 0.0254 #meter
Vol_int = np.pi*(piston_dia/2)**2 #m^2

# FLOW RATES
data['Qa'] = -1 * data['AvPparate']/1e6 * Vol_int #m^3/s
data['Qb'] = data['AvPpbrate']/1e6 * Vol_int #m^3/s

#flow in this exp is from to A to B
#check to make sure we have steady-state flow -- percent difference 
data['QpctDiff'] = (data['Qa'] - data['Qb']) / data['Qa'] * 100 #percent

#calculate an average flow rate from upsteam and downstream
data['Qavg'] = (data['Qa'] + data['Qb'])/2 #m^3/s

# Fracture plane length x thickness 45 x 23.43 = 0.00105435 m^2
A_flow = 0.045 * 0.02343 #m^2

# flow length is 0.04976 m
L_flow = 0.04976 #meter

#viscosity of water at 25 deg. C = 1e-3 Pa.s
nu = 1e-3 #Pa.s

# permeability calculation [m^2]
data['perm'] = (data['Qavg'] * nu * L_flow)/(data['mmPpDiff'] * A_flow) #[m^2]

In [ ]:
# make_runplot(data, y_vars=['ppa press', 'Qa', 'Qb'])

In [ ]:
max_flow_diff_percent = 5
min_flow = 0e-8; # in m^3/s. Minimum flow

flow_diff_idxs = data['QpctDiff'].abs().to_numpy() > max_flow_diff_percent
min_flow_idxs = np.logical_or(data['Qa'].to_numpy() <= min_flow, data['Qb'].to_numpy() <= min_flow)

data['perm'].to_numpy()[flow_diff_idxs] = np.nan
# data['perm'].to_numpy()[min_flow_idxs] = np.nan

In [ ]:
def osc_beg_end(start, osc_freqs):
    cycles = 20
#     osc_freqs = np.array([0.1, 1, 10, 0.1, 1, 10, 1, 1, 1, 1]) # Hz
    rec_freq = 100 #Hz
    hold_time = 60 * rec_freq # rec num

    pp_osc_recs = cycles/osc_freqs * rec_freq # num / Hz * Hz = rec num
    
    pp_start = np.ones(np.size(osc_freqs))*start
    pp_end = np.ones(np.size(osc_freqs))*start + pp_osc_recs[0]
    
    pp_start[1::] = pp_start[1::] + np.cumsum(pp_osc_recs[0:-1] + hold_time)
    pp_end[1::] = pp_end[0::-1] + np.cumsum(pp_osc_recs[1::] + hold_time)
    
    return pp_start.astype(int), pp_end.astype(int)

In [ ]:
def delperm(time_before, time_after, pp_start, pp_end):
    rec_freq = 100 #Hz
    rec_before = time_before * rec_freq #rec num
    rec_after = time_after * rec_freq #rec num
    
    before = pp_start - rec_before
    after = pp_end + rec_after
    
    k0 = np.array(list(map(lambda x: np.nanmean(np.abs(data['perm'])[before[x]:pp_start[x]]), np.arange(np.size(pp_start)))))
    k1 = np.array(list(map(lambda x: np.nanmean(np.abs(data['perm'])[pp_end[x]:after[x]]), np.arange(np.size(pp_end)))))

    delk = (k1-k0)/k0
    
    return delk

In [ ]:
ns10_pp_start, ns10_pp_end = osc_beg_end(230424, osc_freqs = np.array([1, 10, 0.1, 1, 10, 1, 1, 1, 1]))

ns15_pp_start, ns15_pp_end = osc_beg_end(324699, osc_freqs = np.array([0.1, 1, 10, 0.1, 1, 10, 1, 1, 1, 1]))

ns20_pp_start, ns20_pp_end = osc_beg_end(444316, osc_freqs = np.array([0.1, 1, 10, 0.1, 1, 10, 1, 1, 1, 1]))

In [ ]:
ns10_delk = delperm(1, 5, ns10_pp_start, ns10_pp_end)*100
ns15_delk = delperm(1, 5, ns15_pp_start, ns15_pp_end)*100
ns20_delk = delperm(1, 5, ns20_pp_start, ns20_pp_end)*100

In [ ]:
osc_freqs = np.array([0.1, 1, 10, 0.1, 1, 10, 1, 1, 1, 1])
osc_amps = np.array([1, 1, 1, 1, 1, 1, 0.4, 0.8, 0.4, 1])
# [1, 4, 6, 7, 8, 9]

In [ ]:
runs = np.array([[45228,155847],[175000,342500],[342500,504200],
                 [509300,593000],[610100,76700],[768000,859200],
                 [86300,951000],[974700,1137600],[1173400,1136600],
                [],[],[],
                [],[],[],
                [],[],[],[],[],[],[],[]])

In [ ]:
sections = np.array([[45200,504200],[509300,859200],[863000,1136600]])

In [ ]:
from bokeh.plotting import output_file, save

plots = []
p = figure(title='Stresses', tools='pan,box_zoom,undo,save,hover') #, y_axis_type="log" , 
# p.line(data['Time'], data['Hor LOAD'], line_color="crimson", legend_label='NS')
p.line(data['Time'], data['effNS'], line_color="darkred", legend_label='effNS')
p.line(data['Time'], data['ppa press'], line_color='blue', legend_label='Ppa')
# p.circle(data['Time'][ns10_pp_start], data['Ppa LOAD'][ns10_pp_start], line_color='red', fill_color='red', size=6)
# p.circle(data['Time'][ns10_pp_end], data['Ppa LOAD'][ns10_pp_end], line_color='black', fill_color='black', size=6)
p.line(data['Time'], data['ppb press'], line_color="deepskyblue", legend_label='Ppb')
p.yaxis.axis_label = 'Stress (MPa)'
p.legend.location = 'top_left'
p.legend.click_policy='hide'
plots.append(p)

p = figure(x_range=plots[0].x_range, tools='pan,box_zoom,undo,save,hover') # , y_range=(-1e-16,1e-16) , y_axis_type="log"
p.circle(data['Time'], data['perm'].abs(), line_color='black', fill_color='black', size=6)
# # p.circle(data['Time'][ns15_pp_start+pp_osc_recs], data['perm'].abs()[ns15_pp_start+pp_osc_recs], line_color='red', fill_color='black', size=6)
p.xaxis.axis_label = 'Time (s))'
p.yaxis.axis_label = 'Permeability (m^2)'
plots.append(p)
show(gridplot(plots, ncols=1, plot_width=1200, plot_height=400))

# output_file("test.html")
# save(plots)

In [ ]:
from bokeh.plotting import output_file, save

plots = []
p = figure(title='Stresses', x_range=(11480, 17000), tools='pan,box_zoom,undo,save,hover') #, y_axis_type="log" , 
# p.line(data['Time'], data['Hor LOAD'], line_color="crimson", legend_label='NS')
# p.line(data['Time'], data['effNS'], line_color="darkred", legend_label='effNS')
p.line(data['Time'], data['Ppa LOAD'], line_color='blue', legend_label='Ppa')
# p.circle(data['Time'][ns10_pp_start], data['Ppa LOAD'][ns10_pp_start], line_color='red', fill_color='red', size=6)
# p.circle(data['Time'][ns10_pp_end], data['Ppa LOAD'][ns10_pp_end], line_color='black', fill_color='black', size=6)
# p.line(data['Time'], data['Ppb LOAD'], line_color="deepskyblue", legend_label='Ppb')
p.yaxis.axis_label = 'Stress (MPa)'
p.legend.location = 'top_left'
p.legend.click_policy='hide'
plots.append(p)

p = figure(x_range=plots[0].x_range, y_range=(-2.5e-9,3.5e-9), tools='pan,box_zoom,undo,save,hover') # , y_range=(-1e-16,1e-16) , y_axis_type="log"
p.line(data['Time'], data['Qa'], line_color='black')
# p.circle(data['Time'][ns15_pp_start+pp_osc_recs], data['perm'].abs()[ns15_pp_start+pp_osc_recs], line_color='red', fill_color='black', size=6)
p.xaxis.axis_label = 'Time (s))'
p.yaxis.axis_label = 'Qa (m^3/s)'
plots.append(p)
show(gridplot(plots, ncols=1, plot_width=1200, plot_height=400))

# output_file("test.html")
# save(plots)

In [ ]:
plots = []
p = figure(tools='pan,box_zoom,undo,save,hover') #, y_axis_type="log" , 
p.circle(osc_amps[np.array([0, 3, 5, 6, 7, 8])], ns10_delk[np.array([0, 3, 5, 6, 7, 8])], legend_label='10MPa', line_color='dodgerblue', fill_color='dodgerblue', size=8)
p.circle(osc_amps[np.array([1, 4, 6, 7, 8, 9])], ns15_delk[np.array([1, 4, 6, 7, 8, 9])], legend_label='15MPa', line_color='blue', fill_color='blue', size=8)
p.circle(osc_amps[np.array([1, 4, 6, 7, 8, 9])], ns20_delk[np.array([1, 4, 6, 7, 8, 9])], legend_label='20MPa', line_color='darkblue', fill_color='darkblue', size=8)
p.legend.click_policy='hide'
p.legend.location='top_left'
p.xaxis.axis_label = 'Oscillation Amp. (MPa)'
p.yaxis.axis_label = 'del k/k0 (%)'
plots.append(p)
show(gridplot(plots, ncols=1, plot_width=600, plot_height=600))